In [1]:
import pandas as pd
import numpy as np

In [ ]:
def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1eTyWC2ED8frOBNy7r2qN_hTpSwZC1jYZ/view?usp=drive_link"

direct_link = get_direct_link(link)
df = pd.read_csv(direct_link)

In [3]:
df['date'] = pd.to_datetime(df.date)
df = df.sort_values(by='date')

In [8]:
def wikibuzz(winner_pageviews, winner_median, loser_pageviews, loser_median):
    try:
        buzz = np.log(winner_pageviews / winner_median) - np.log(loser_pageviews / loser_median)
    except ZeroDivisionError:
        buzz = 'ZeroDivisionError'
    return buzz

df['wikibuzz_r'] = [wikibuzz(row[0], row[1], row[2], row[3]) for row in zip(df['wiki_yesterday_r'], df['wiki_med365_r'],
                                                                                df['wiki_yesterday_b'], df['wiki_med365_b'])]
df['wikibuzz_b'] = [wikibuzz(row[0], row[1], row[2], row[3]) for row in zip(df['wiki_yesterday_b'], df['wiki_med365_b'],
                                                                                df['wiki_yesterday_r'], df['wiki_med365_r'])]

C:\Users\Archie Stevens\AppData\Local\Temp\ipykernel_41572\1175357422.py:3: RuntimeWarning: divide by zero encountered in log
  buzz = np.log(winner_pageviews / winner_median) - np.log(loser_pageviews / loser_median)
C:\Users\Archie Stevens\AppData\Local\Temp\ipykernel_41572\1175357422.py:3: RuntimeWarning: divide by zero encountered in log
  buzz = np.log(winner_pageviews / winner_median) - np.log(loser_pageviews / loser_median)


In [9]:
df['inverse_odds_r'] = [1/x for x in df['R_odds']]
df['inverse_odds_b'] = [1/x for x in df['B_odds']]

In [10]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')

In [11]:
df['match_id'] = df.index

df_red = df[['match_id', 'R_fighter', 'date', 'wikibuzz_r', 'inverse_odds_r', 'Winner']].copy()
df_blue = df[['match_id', 'B_fighter', 'date', 'wikibuzz_b', 'inverse_odds_b', 'Winner']].copy()

In [12]:
outcome_mapping_r = {'Red': 1, 'Blue': 0}
outcome_mapping_b = {'Red': 0, 'Blue': 1}

In [13]:
df_red['outcome'] = df_red['Winner'].map(outcome_mapping_r)
df_blue['outcome'] = df_blue['Winner'].map(outcome_mapping_b)

In [14]:
df_red.rename({'R_fighter': 'fighter', 'wikibuzz_r': 'wikibuzz','inverse_odds_r': 'inverse_odds'}, axis=1, inplace=True)
df_blue.rename({'B_fighter': 'fighter', 'wikibuzz_b': 'wikibuzz','inverse_odds_b': 'inverse_odds'}, axis=1, inplace=True)

In [15]:
df = pd.concat([df_red, df_blue], axis=0)

In [17]:
df = df.loc[df["wikibuzz"] != np.inf]
df = df.loc[df["wikibuzz"] != -np.inf]
df = df.loc[df["wikibuzz"] != 'ZeroDivisionError']

df = df[df['wikibuzz'].notna()]
df = df[df['inverse_odds'].notna()]

df.sort_values(by='date', inplace = True)

In [21]:
df.to_csv('ufc_final.csv', index=False)